### Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# additional imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Loading API

In [2]:
# Load API Credentials
with open('/Users/paull/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### Search Terms

In [3]:
# set our API call parameters 
LOCATION = 'Portland,OR'
TERM = 'Bagels'

### JSON file name

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_OR_bagels.json"
JSON_FILE

'Data/results_in_progress_OR_bagels.json'

### Check for previous file

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_OR_bagels.json already exists.


### Count previous results

In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 20 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

### How many results total

In [8]:
## How many results total?
total_results = results['total']
total_results



670

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

### Math function to calculate pages

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



33

### Add new results to old results

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


### function

In [12]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # delete file and confirm it no longer exists
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")

    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)

        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

In [13]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[!] Data/results_in_progress_OR_bagels.json already exists. Deleting previous file...
[i] Data/results_in_progress_OR_bagels.json not found. Saving empty list to new file.
- 0 previous results found.


34

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/34 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,9l2YZ9KDnIOdBzhq0pW93Q,bernsteins-bagels-portland-2,Bernstein's Bagels,https://s3-media3.fl.yelpcdn.com/bphoto/qrZ7mi...,False,https://www.yelp.com/biz/bernsteins-bagels-por...,114,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 45.5408365116567, 'longitude': -1...",[delivery],$,"{'address1': '816 N Russell St', 'address2': '...",+15038831222,(503) 883-1222,1636.514808
1,huErclb5v4o1Ld9_9hZXDg,bentley-s-bagels-portland,Bentley’s Bagels,https://s3-media2.fl.yelpcdn.com/bphoto/Qc0YVn...,False,https://www.yelp.com/biz/bentley-s-bagels-port...,86,"[{'alias': 'bagels', 'title': 'Bagels'}]",4.5,"{'latitude': 45.52945, 'longitude': -122.69432}",[delivery],NaN,"{'address1': '918 Nw 21st Ave', 'address2': No...",+19713027019,(971) 302-7019,3467.074749
2,I8Krk_FEXbWNqv26Ny7W8w,henry-higgins-boiled-bagels-portland-8,Henry Higgins Boiled Bagels,https://s3-media2.fl.yelpcdn.com/bphoto/_1KbHy...,False,https://www.yelp.com/biz/henry-higgins-boiled-...,147,"[{'alias': 'bagels', 'title': 'Bagels'}]",4.0,"{'latitude': 45.5268171305383, 'longitude': -1...",[delivery],$,"{'address1': '523 NE 19th Ave', 'address2': ''...",+19719400256,(971) 940-0256,1850.571594
3,qHUy0Xr9MsytU3FAnh8Rwg,just-bob-portland-2,Just Bob.,https://s3-media1.fl.yelpcdn.com/bphoto/B7Riv1...,False,https://www.yelp.com/biz/just-bob-portland-2?a...,170,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,"{'latitude': 45.55925, 'longitude': -122.64074}",[delivery],$,"{'address1': '2403 NE Alberta St', 'address2':...",+15034674656,(503) 467-4656,2101.342040
4,lnBtjouBKqOUl9Oo0VD5mQ,henry-higgins-boiled-bagels-portland-5,Henry Higgins Boiled Bagels,https://s3-media3.fl.yelpcdn.com/bphoto/7_ar5S...,False,https://www.yelp.com/biz/henry-higgins-boiled-...,320,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 45.48914, 'longitude': -122.59622}",[delivery],$,"{'address1': '6420 SE Foster Rd', 'address2': ...",+19712718613,(971) 271-8613,7447.348509


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
665,BbZywCVU9j5HuBEgvDPtoA,safeway-king-city-4,Safeway,https://s3-media2.fl.yelpcdn.com/bphoto/KIFF-V...,False,https://www.yelp.com/biz/safeway-king-city-4?a...,62,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 45.40724, 'longitude': -122.79270...",[],$$,"{'address1': '15570 SW Pacific Hwy', 'address2...",+15035986001,(503) 598-6001,18540.061128
666,LfbxX1KrfIl5yldQ6bxMKw,wasabi-sushi-pdx-happy-valley,Wasabi Sushi PDX,https://s3-media2.fl.yelpcdn.com/bphoto/VcBhr3...,False,https://www.yelp.com/biz/wasabi-sushi-pdx-happ...,144,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.0,"{'latitude': 45.42434, 'longitude': -122.51527}","[delivery, pickup]",$$,"{'address1': '13551 SE 145th Ave', 'address2':...",+15037245199,(503) 724-5199,17029.444068
667,mi1dZbnxwYPM3eLrWM-6MA,starbucks-hillsboro-5,Starbucks,https://s3-media1.fl.yelpcdn.com/bphoto/E5dp46...,False,https://www.yelp.com/biz/starbucks-hillsboro-5...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 45.5337682, 'longitude': -122.935...",[delivery],$$,"{'address1': '1221 NE 48th Ave', 'address2': '...",+15036931239,(503) 693-1239,21910.915686
668,HFAcdeRwBohKst5Nlsaijg,starbucks-beaverton-14,Starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/xK2t24...,False,https://www.yelp.com/biz/starbucks-beaverton-1...,45,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 45.438221, 'longitude': -122.8268...",[delivery],$$,"{'address1': '14600 SW Murray Scholls Dr', 'ad...",+15035907886,(503) 590-7886,17784.965610
669,XUuzSaINm3BylXMnLylfWg,safeway-beaverton-2,Safeway,https://s3-media2.fl.yelpcdn.com/bphoto/OTBCxe...,False,https://www.yelp.com/biz/safeway-beaverton-2?a...,84,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 45.44206144957988, 'longitude': -...",[],$$,"{'address1': '14555 SW Teal Blvd', 'address2':...",+15035909755,(503) 590-9755,17508.110946


In [17]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



0

In [18]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [19]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_OR_bagels.csv.gz', compression='gzip',index=False)
